In [11]:
import json
import sys
from functools import partial
import copy
from torch.autograd import Variable

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
import tqdm
import wandb
from ignite.contrib.handlers import wandb_logger
from ignite.engine import (Engine, Events, create_supervised_evaluator,
                           create_supervised_trainer)
from ignite.handlers import ModelCheckpoint
from ignite.handlers.param_scheduler import LRScheduler
from ignite.metrics import Accuracy, Loss
from scipy.io.arff import loadarff
from sklearn.model_selection import train_test_split
from torch import nn
from torch.functional import F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

sys.path.append('../')
from attacks.deepfool import deepfool
from src.datasets import FordDataset
from src.models import TransformerClassification
from src.utils import build_optimizer, str2torch

In [12]:
with open('../configs/transformer_87.json') as f:
    config =  json.load(f)
config['train']['optimizer'] = str2torch(config['train']['optimizer'])

In [13]:
test_path = "../data/FordA/FordA_TEST.arff"
test_dataset = FordDataset(test_path, config['data'])
test_dataloader = DataLoader(test_dataset, batch_size=64)

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TransformerClassification(config).to(device)
weights = torch.load(r'C:\Users\ptmeg\OneDrive\Документы\Skoltech\Term3\ML\ts_robustness\models\best_model_29_accuracy=0.8868.pt')
model.load_state_dict(weights)

<All keys matched successfully>

In [15]:
def validation_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch[0].to(device), batch[1].to(device)
        y_pred = model(x)
        return y_pred, y

test_evaluator = Engine(validation_step)

# Attach metrics to the evaluators
metrics = {
    'accuracy': Accuracy(output_transform=lambda x: (torch.argmax(x[0], dim=1), x[1])),
}

for name, metric in metrics.items():
    metric.attach(test_evaluator, name)

In [16]:
test_evaluator.run(test_dataloader)
test_evaluator.state.metrics['accuracy']

0.884185606060606

In [7]:
iters = []
for i in tqdm.tqdm(range(len(test_dataset))):
    if i == 20: break
    test_sample = torch.from_numpy(test_dataset[i][0]).unsqueeze(1).to(device)
    r_tot, loop_i, label, k_i, pert_image = deepfool(test_sample, model, 2, 0.2, max_iter=30, device=device)
    iters.append(loop_i)
    # if i == 100:
    #     break

  0%|          | 20/10560 [00:00<02:54, 60.24it/s]


In [8]:
unique, counts = np.unique(iters, return_counts=True)

In [9]:
unique, counts

(array([1, 2, 3, 4]), array([6, 8, 3, 3], dtype=int64))

In [10]:
np.mean(iters), np.std(iters)

(2.15, 1.0136567466356647)